Домашнее задание №1

Данные:
https://www.kaggle.com/datasets/gauravduttakiit/loan-defaulter/code?datasetId=807638&sortBy=voteCount

Задача: 
(Колонка target) Выявление клиентов, у которых возникают трудности с кредитом (target == 1). 

Критерии оценки, что должно быть:
- Загрузка данных
- Базовое EDA
- Подготовка (аномалии, пропуски, стандартизация, train/test split)
- Обучение алгоритма
- Метрики качества (Максимизируем f1 macro avg)

Алгоритм, базовый:
- Нейросеть из >= 1 nn.Linear скрытых слоев с использованием torch (за основу можно взять ноутбук занятия или nn.sequential)

Алгоритм продвинутый (вместо базового, если есть желание на отлично):
- Даталоадер + все что в базовом или другие типы слоев + дропаут + батчнорм + AdamW + инициализация весов

Для тех, кто чувствует, что задача слишком простая - можно взять любой из датасетов кегл для глубокого обучения и решить ее. Важно, чтоб проделанная работа была по сложности >= продвинутого задания (желательно брать не слишком известный датасет). В письме отметьте, что вы изменили набор данных, и дайте на него ссылку.

Присылать дз можно как ссылкой на колаб (предпочтительнее) так и ноутбуком. Альтернативно - гит/кеггл, где в описании аккаунта указаны ваши инициалы.

Ваши решения присылать ссылкой на ноутбук на почту mlsber.fall21@gmail.com (с пометкой "Про" в теме письма)

ВАЖНО:
* В шапке укажите ваше ФИО и номер задания.
* Когда выбираете "поделиться" в колабе нажимайте "изменить" и выставляйте "доступные пользователям у которых есть ссылка" - иначе у нас нет прав на просмотр вашего решения.

In [1]:
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [2]:
!unzip /drive/MyDrive/ml/dl_course_2022/dl_01.zip -d /content

In [3]:
import pandas as pd

In [4]:
data = pd.read_csv('/content/application_data.csv')

In [5]:
data = data.drop(columns=['SK_ID_CURR'])

In [6]:
# get numeric columns
columns_num = list(data.describe().columns)
columns_num = columns_num[1:]

In [7]:
# check numeric data
for col in columns_num: 
    print(data[col].describe().T)

count    307511.000000
mean          0.417052
std           0.722121
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max          19.000000
Name: CNT_CHILDREN, dtype: float64
count    3.075110e+05
mean     1.687979e+05
std      2.371231e+05
min      2.565000e+04
25%      1.125000e+05
50%      1.471500e+05
75%      2.025000e+05
max      1.170000e+08
Name: AMT_INCOME_TOTAL, dtype: float64
count    3.075110e+05
mean     5.990260e+05
std      4.024908e+05
min      4.500000e+04
25%      2.700000e+05
50%      5.135310e+05
75%      8.086500e+05
max      4.050000e+06
Name: AMT_CREDIT, dtype: float64
count    307499.000000
mean      27108.573909
std       14493.737315
min        1615.500000
25%       16524.000000
50%       24903.000000
75%       34596.000000
max      258025.500000
Name: AMT_ANNUITY, dtype: float64
count    3.072330e+05
mean     5.383962e+05
std      3.694465e+05
min      4.050000e+04
25%      2.385000e+05
50%      4.500000e+05
75%    

In [8]:
# get categorical columns
columns_cat = data.select_dtypes(include=['object']).columns

In [9]:
data_num = data.drop(columns=columns_cat)

In [10]:
# remove outliers
import numpy as np

def cap_data(df, columns_num):
    for col in columns_num:
        if (((df[col].dtype)=='float64') | ((df[col].dtype)=='int64')):
            percentiles = df[col].quantile([0.25,0.75]).values
            df[col][df[col] <= percentiles[0]] = percentiles[0]
            df[col][df[col] >= percentiles[1]] = percentiles[1]
        else:
            df[col]=df[col]
    return df

data_num = cap_data(data_num, columns_num)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [11]:
data_num = data_num.fillna(data_num.mean())

In [12]:
# standartization
from sklearn import preprocessing

x = data_num.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
data_num_norm = pd.DataFrame(x_scaled, columns=data_num.columns)

In [13]:
data_num_norm.isnull().values.any()

False

In [14]:
# get categorical data
data_cat = data.select_dtypes(include=['object']).copy()

In [15]:
data_ohe = pd.get_dummies(data_cat, columns=columns_cat)
data_ohe = data_ohe.fillna(0)

In [16]:
data = pd.concat([data_num_norm, data_ohe], axis=1)

In [17]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
TARGET,307511.0,0.080729,0.272419,0.0,0.0,0.000000,0.0,1.0
CNT_CHILDREN,307511.0,0.299632,0.458097,0.0,0.0,0.000000,1.0,1.0
AMT_INCOME_TOTAL,307511.0,0.459063,0.415638,0.0,0.0,0.385000,1.0,1.0
AMT_CREDIT,307511.0,0.480072,0.408237,0.0,0.0,0.452114,1.0,1.0
AMT_ANNUITY,307511.0,0.485424,0.403203,0.0,0.0,0.463645,1.0,1.0
...,...,...,...,...,...,...,...,...
WALLSMATERIAL_MODE_Panel,307511.0,0.214757,0.410654,0.0,0.0,0.000000,0.0,1.0
"WALLSMATERIAL_MODE_Stone, brick",307511.0,0.210773,0.407858,0.0,0.0,0.000000,0.0,1.0
WALLSMATERIAL_MODE_Wooden,307511.0,0.017437,0.130892,0.0,0.0,0.000000,0.0,1.0
EMERGENCYSTATE_MODE_No,307511.0,0.518446,0.499660,0.0,0.0,1.000000,1.0,1.0


In [18]:
# train-valid-test split
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)
valid, test = train_test_split(test, test_size=0.5)

In [19]:
X_train = np.array(train.drop(columns=['TARGET']), dtype=np.float32)
y_train = np.array(train['TARGET'], dtype=np.float32)

X_valid = np.array(valid.drop(columns=['TARGET']), dtype=np.float32)
y_valid = np.array(valid['TARGET'], dtype=np.float32)

X_test = np.array(test.drop(columns=['TARGET']), dtype=np.float32)
y_test = np.array(test['TARGET'], dtype=np.float32)

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import f1_score, accuracy_score, precision_score

In [21]:
class CustomDataset(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [22]:
train_data = CustomDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))

In [23]:
# class imbalance
import numpy as np
from torch.utils.data.sampler import WeightedRandomSampler

counts = np.bincount(y_train.astype(int))
labels_weights = 1. / counts
weights = labels_weights[y_train.astype(int)]
sampler = WeightedRandomSampler(weights, len(weights))

In [24]:
train_loader = DataLoader(dataset=train_data, sampler=sampler, batch_size=1000)

In [25]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [37]:
class SimleNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimleNetwork,self).__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim * 2)
        self.linear2 = nn.Linear(hidden_dim * 2, hidden_dim)
        self.out = nn.Linear(hidden_dim, output_dim)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(hidden_dim * 2)
        self.batchnorm2 = nn.BatchNorm1d(hidden_dim)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.batchnorm1(x)
        x = self.linear2(x)
        x = self.relu(x)
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.out(x)
        return x

In [38]:
input_dim = X_train.shape[1]
hidden_dim = 200
output_dim = 1

In [39]:
loss_fn = nn.BCEWithLogitsLoss()

In [40]:
model = SimleNetwork(input_dim, hidden_dim, output_dim)

In [41]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

model.apply(init_weights)

SimleNetwork(
  (linear1): Linear(in_features=244, out_features=400, bias=True)
  (linear2): Linear(in_features=400, out_features=200, bias=True)
  (out): Linear(in_features=200, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [42]:
model.to(device)

SimleNetwork(
  (linear1): Linear(in_features=244, out_features=400, bias=True)
  (linear2): Linear(in_features=400, out_features=200, bias=True)
  (out): Linear(in_features=200, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [43]:
optimizer = optim.AdamW(model.parameters(), lr=0.001)

In [44]:
def eval(model, x, y):
    model.eval()
    with torch.no_grad():
        X = torch.FloatTensor(x)
        y = torch.FloatTensor(y)
        X = X.to(device)
        y_test_pred = model(X)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_test_pred = torch.round(y_test_pred)
        y_test_pred = y_test_pred.reshape(1, -1).cpu().detach().numpy()[0]

    return y_test_pred

In [45]:
for epoch in range(100):
    model.train()
    for data in train_loader:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = torch.sigmoid(model(inputs))
        loss = loss_fn(outputs, labels.reshape(-1, 1))
        loss.backward()
        optimizer.step()

    predictions = eval(model, X_valid, y_valid)
    print(epoch, f1_score(y_valid.tolist(), predictions.tolist(), average='macro'))

0 0.584393387496443
1 0.587633660320497
2 0.5854317538886462
3 0.5696220368605227
4 0.586167955233194
5 0.5804593392884786
6 0.5704101698228974
7 0.5803976564447768
8 0.5828915245927893
9 0.5797927934947389
10 0.577558934895578
11 0.5758985699828616
12 0.5795603206290724
13 0.5830091062161087
14 0.5810385963383167
15 0.578276822324775
16 0.5779750049883505
17 0.5857446493632626
18 0.579774265462413
19 0.5811121227980872
20 0.5794029997432838
21 0.5802375777691827
22 0.5805279765240997
23 0.5804399108852832
24 0.5804798120845962
25 0.5777824843490276
26 0.5791565927121638
27 0.5728722437265735
28 0.5750502072888616
29 0.5774144592691769
30 0.576222017843757
31 0.5742394520555477
32 0.5774865218262637
33 0.5751932251313552
34 0.5701825172186669
35 0.5759634912161231
36 0.5776858273836627
37 0.5711643822939788
38 0.5742669473925983
39 0.5691091277759235
40 0.5708227947624054
41 0.5717003129238463
42 0.5710886030116551
43 0.5750185155177251
44 0.5751810027256137
45 0.5737861674984324
46 0.

In [46]:
predictions = eval(model, X_test, y_test)
print(f1_score(y_test.tolist(), predictions.tolist(), average='macro'))
print(accuracy_score(y_test.tolist(), predictions.tolist()))
print(precision_score(y_test.tolist(), predictions.tolist()))

0.5564687307551927
0.8433922996878251
0.16815892557358703
